In [1]:
import pandas as pd
import os

In [2]:
school_data_to_load = os.path.join("resources","schools_complete.csv")
students_data_to_load = os.path.join("resources","students_complete.csv")
# Add each prefix and suffix to remove to a list.
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

In [3]:
school_data_df = pd.read_csv(school_data_to_load)
#school_data_df.head()

In [4]:
# Determine if there are any missing values in the school data.
#school_data_df.count()
#school_data_df.notnull()

In [5]:
#Read student data file and determine if there are any missing values
student_data_df = pd.read_csv(students_data_to_load)
#student_data_df.count()
#student_data_df.isnull().sum()

In [6]:
# Iterate through the words in the "prefixes_suffixes" list and replace them with an empty space, "".
for word in prefixes_suffixes:
    student_data_df["student_name"] = student_data_df["student_name"].str.replace(word,"")
#student_data_df.head()    

In [7]:
clean_students_data_to_load = os.path.join("resources","clean_students_complete.csv")
clean_students_data_df = pd.read_csv(clean_students_data_to_load)

In [8]:
def calculatepassingpercentage(passing_count,total_count):
    return (passing_count / float(total_count)*100)

In [9]:
complete_student_data_df = pd.merge(school_data_df,clean_students_data_df, on=["school_name","school_name"])

In [10]:
total_school_count = school_data_df.school_name.count()

In [11]:
total_student_count = clean_students_data_df["Student ID"].count()

In [12]:
total_budget = school_data_df["budget"].sum()

In [13]:
average_math_score = clean_students_data_df["math_score"].mean()
averrage_reading_score = clean_students_data_df["reading_score"].mean()

In [14]:
passing_math_score = complete_student_data_df[complete_student_data_df["math_score"] >= 70]
passing_reading_score = complete_student_data_df[complete_student_data_df["reading_score"] >= 70]
overall_reading_score = complete_student_data_df[(complete_student_data_df["math_score"] >= 70)  & (complete_student_data_df["reading_score"] >= 70)]
#overall_reading_score.head(10)

In [15]:
passing_math_percentage = calculatepassingpercentage(passing_math_score["student_name"].count(),total_student_count)
passing_reading_percentage = calculatepassingpercentage(passing_reading_score["student_name"].count(),total_student_count)
passing_overall_percentage = calculatepassingpercentage(overall_reading_score["student_name"].count(),total_student_count)

In [16]:
district_summary_df = pd.DataFrame([{
    "Total Schools": total_school_count,
    "Total Students": ("{:,}".format(total_student_count)),
    "Total Budget": ("${:,.2f}".format(total_budget)),
    "Average Math Score": ("{:.1f}".format(average_math_score)),
    "Average Reading Score": ("{:.1f}".format(averrage_reading_score)),
    "% Passing Math": ("{:.0f}".format(passing_math_percentage)),
    "% Passing Reading": ("{:.0f}".format(passing_reading_percentage)),
    "% Overall Passing": ("{:.0f}".format(passing_overall_percentage)),
}])
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",79.0,81.9,75,86,65


In [17]:
#Starting section 4.8
#Determine the school type
per_school_types = school_data_df.set_index(["school_name"])["type"]
df = pd.DataFrame(per_school_types)

In [18]:
per_school_student_counts = school_data_df.set_index(["school_name"])["size"]
per_school_student_counts

school_name
Huang High School        2917
Figueroa High School     2949
Shelton High School      1761
Hernandez High School    4635
Griffin High School      1468
Wilson High School       2283
Cabrera High School      1858
Bailey High School       4976
Holden High School        427
Pena High School          962
Wright High School       1800
Rodriguez High School    3999
Johnson High School      4761
Ford High School         2739
Thomas High School       1635
Name: size, dtype: int64

In [19]:
# Calculate the total student count.
#per_school_counts = complete_student_data_df["school_name"].value_counts()
#per_school_counts

In [20]:
per_school_budget = school_data_df.set_index(["school_name"])["budget"]
#per_school_budget

In [21]:
per_school_capita = per_school_budget / per_school_student_counts
#per_school_capita

In [22]:
#per_school_averages = complete_student_data_df.groupby(["school_name"]).mean()
per_school_math = complete_student_data_df.groupby(["school_name"]).mean()["math_score"]
per_school_read = complete_student_data_df.groupby(["school_name"]).mean()["reading_score"]
#per_school_math

In [23]:
per_school_passing_math_score = complete_student_data_df[complete_student_data_df["math_score"] >= 70]
per_school_passing_reading_score = complete_student_data_df[complete_student_data_df["reading_score"] >= 70]
per_school_overall_passing_score = complete_student_data_df[(complete_student_data_df["math_score"] >= 70) & (complete_student_data_df["reading_score"] >= 70)]

In [24]:
per_school_passing_math_score = per_school_passing_math_score.groupby(["school_name"]).count()["student_name"]
per_school_passing_reading_score = per_school_passing_reading_score.groupby(["school_name"]).count()["student_name"]
per_school_overall_passing_score = per_school_overall_passing_score.groupby(["school_name"]).count()["student_name"]

In [25]:
# Calculate the percentage of passing math and reading scores per school.
per_school_passing_math = per_school_passing_math_score / per_school_student_counts * 100
per_school_passing_reading = per_school_passing_reading_score / per_school_student_counts * 100
per_school_passing_overall = per_school_overall_passing_score / per_school_student_counts * 100
#per_school_passing_overall

In [26]:
per_school_summary_df = pd.DataFrame({
             "School Type": per_school_types,
             "Total Students": per_school_student_counts,
             "Total School Budget": per_school_budget.map("${:,.2f}".format),
             "Per Student Budget": per_school_capita.map("${:,.2f}".format),
             "Average Math Score": per_school_math,
           "Average Reading Score": per_school_read,
           "% Passing Math": per_school_passing_math,
           "% Passing Reading": per_school_passing_reading,
           "% Overall Passing": per_school_passing_overall})
per_school_summary_df = per_school_summary_df[["School Type","Total Students","Total School Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]]
per_school_summary_df.head()

,School Type,Total Students,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",83.351499,83.816757,93.392371,97.138965,90.599455


In [27]:
#sort and show top five schools
top_schools = per_school_summary_df.sort_values(["% Overall Passing"],ascending=False)
top_schools = top_schools[["School Type","Total Students","Total School Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]]
top_schools.head()

,School Type,Total Students,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",83.839917,84.044699,94.594595,95.945946,90.540541


In [28]:
top_schools.tail()

,School Type,Total Students,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Johnson High School,District,4761,"$3,094,650.00",77.072464,80.966394,66.057551,81.222432,53.539172
Hernandez High School,District,4635,"$3,022,020.00",77.289752,80.934412,66.752967,80.862999,53.527508
Huang High School,District,2917,"$1,910,635.00",76.629414,81.182722,65.683922,81.316421,53.513884
Figueroa High School,District,2949,"$1,884,411.00",76.711767,81.158020,65.988471,80.739234,53.204476
Rodriguez High School,District,3999,"$2,547,363.00",76.842711,80.744686,66.366592,80.220055,52.988247


In [29]:
ninth_graders = complete_student_data_df[(complete_student_data_df["grade"] == "9th")]
tenth_graders = complete_student_data_df[(complete_student_data_df["grade"] == "10th")]
eleventh_graders = complete_student_data_df[(complete_student_data_df["grade"] == "11th")]
twelfth_graders = complete_student_data_df[(complete_student_data_df["grade"] == "12th")]

In [30]:
ninth_graders_avg_math_score = ninth_graders.groupby(["school_name"]).mean()["math_score"]
tenth_graders_avg_math_score = tenth_graders.groupby(["school_name"]).mean()["math_score"]
eleventh_graders_avg_math_score = eleventh_graders.groupby(["school_name"]).mean()["math_score"]
twelfth_graders_avg_math_score = twelfth_graders.groupby(["school_name"]).mean()["math_score"]

In [31]:
ninth_graders_avg_reading_score = ninth_graders.groupby(["school_name"]).mean()["reading_score"]
tenth_graders_avg_reading_score = tenth_graders.groupby(["school_name"]).mean()["reading_score"]
eleventh_graders_avg_reading_score = eleventh_graders.groupby(["school_name"]).mean()["reading_score"]
twelfth_graders_avg_reading_score = twelfth_graders.groupby(["school_name"]).mean()["reading_score"]

In [32]:
# Combine each Series for average math scores by school into single DataFrame.
math_scores_by_grade = pd.DataFrame({
               "9th": ninth_graders_avg_math_score.map("{:,.1f}".format),
               "10th": tenth_graders_avg_math_score.map("{:,.1f}".format),
               "11th": eleventh_graders_avg_math_score.map("{:,.1f}".format),
               "12th": twelfth_graders_avg_math_score.map("{:,.1f}".format)})

math_scores_by_grade = math_scores_by_grade[
                 ["9th", "10th", "11th", "12th"]]

# Remove the index name.
math_scores_by_grade.index.name = None
# Display the DataFrame.
math_scores_by_grade.head()

,9th,10th,11th,12th
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4


In [33]:
# Combine each Series for average reading scores by school into single DataFrame.
reading_scores_by_grade = pd.DataFrame({
              "9th": ninth_graders_avg_reading_score.map("{:.1f}".format),
              "10th": tenth_graders_avg_reading_score.map("{:.1f}".format),
              "11th": eleventh_graders_avg_reading_score.map("{:.1f}".format),
              "12th": twelfth_graders_avg_reading_score.map("{:.1f}".format)})

reading_scores_by_grade = reading_scores_by_grade[
                 ["9th", "10th", "11th", "12th"]]

# Remove the index name.
reading_scores_by_grade.index.name = None
# Display the DataFrame.
reading_scores_by_grade.head()

,9th,10th,11th,12th
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0


In [34]:
# Get the descriptive statistics for the per_school_capita.
per_school_capita.describe()

count     15.000000
mean     620.066667
std       28.544368
min      578.000000
25%      591.500000
50%      628.000000
75%      641.500000
max      655.000000
dtype: float64

In [35]:
spending_bins = [0,585,630,645,675]
per_school_capita.groupby(pd.cut(per_school_capita, spending_bins)).count()
group_names = ["<$584","$585-629","$630-644","$645-675"]

In [36]:
# Categorize spending based on the bins.
per_school_summary_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=group_names)

per_school_summary_df

,School Type,Total Students,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
Bailey High School,District,4976,"$3,124,928.00",77.048432,81.033963,66.680064,81.933280,54.642283,$585-629
Cabrera High School,Charter,1858,"$1,081,356.00",83.061895,83.975780,94.133477,97.039828,91.334769,<$584
Figueroa High School,District,2949,"$1,884,411.00",76.711767,81.158020,65.988471,80.739234,53.204476,$630-644
Ford High School,District,2739,"$1,763,916.00",77.102592,80.746258,68.309602,79.299014,54.289887,$630-644
Griffin High School,Charter,1468,"$917,500.00",83.351499,83.816757,93.392371,97.138965,90.599455,$585-629
Hernandez High School,District,4635,"$3,022,020.00",77.289752,80.934412,66.752967,80.862999,53.527508,$645-675
Holden High School,Charter,427,"$248,087.00",83.803279,83.814988,92.505855,96.252927,89.227166,<$584
Huang High School,District,2917,"$1,910,635.00",76.629414,81.182722,65.683922,81.316421,53.513884,$645-675
Johnson High School,District,4761,"$3,094,650.00",77.072464,80.966394,66.057551,81.222432,53.539172,$645-675
Pena High School,Charter,962,"$585,858.00",83.839917,84.044699,94.594595,95.945946,90.540541,$585-629


In [37]:
avg_spending_math_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
avg_spending_reading_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
avg_spending_passing_math_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
avg_spending_passing_reading_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
avg_spending_overall_passing_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

In [38]:
# Assemble into DataFrame.
spending_summary_df = pd.DataFrame({
          "Average Math Score" : avg_spending_math_scores.map("{:.1f}".format),
          "Average Reading Score": avg_spending_reading_scores.map("{:.1f}".format),
          "% Passing Math": avg_spending_passing_math_scores.map("{:01f}".format),
          "% Passing Reading": avg_spending_passing_reading_scores.map("{:.0f}".format),
          "% Overall Passing": avg_spending_overall_passing_scores.map("{:.0f}".format)})

spending_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.5,83.9,93.460096,97,90
$585-629,81.9,83.2,87.133538,93,81
$630-644,78.5,81.6,73.484209,84,63
$645-675,77.0,81.0,66.164813,81,54


In [39]:
per_school_student_counts.describe()

count      15.000000
mean     2611.333333
std      1420.915282
min       427.000000
25%      1698.000000
50%      2283.000000
75%      3474.000000
max      4976.000000
Name: size, dtype: float64

In [40]:
school_size_bins = [0,1000,2000,5000]
bin_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [41]:
per_school_summary_df["School Size"] = pd.cut(per_school_summary_df["Total Students"], school_size_bins, labels=bin_names)

per_school_summary_df.head()

,School Type,Total Students,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student),School Size
Bailey High School,District,4976,"$3,124,928.00",77.048432,81.033963,66.680064,81.933280,54.642283,$585-629,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",83.061895,83.975780,94.133477,97.039828,91.334769,<$584,Medium (1000-2000)
Figueroa High School,District,2949,"$1,884,411.00",76.711767,81.158020,65.988471,80.739234,53.204476,$630-644,Large (2000-5000)
Ford High School,District,2739,"$1,763,916.00",77.102592,80.746258,68.309602,79.299014,54.289887,$630-644,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",83.351499,83.816757,93.392371,97.138965,90.599455,$585-629,Medium (1000-2000)


In [42]:
avg_school_size_math_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Math Score"]
avg_school_size_reading_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Reading Score"]
avg_school_size_passing_math_scores = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Math"]
avg_school_size_passing_reading_scores = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Reading"]
avg_school_size_overall_passing_scores = per_school_summary_df.groupby(["School Size"]).mean()["% Overall Passing"]

In [43]:
# Assemble into DataFrame.
school_size_summary_df = pd.DataFrame({
          "Average Math Score" : avg_school_size_math_scores.map("{:.1f}".format),
          "Average Reading Score": avg_school_size_reading_scores.map("{:.1f}".format),
          "% Passing Math": avg_school_size_passing_math_scores.map("{:.0f}".format),
          "% Passing Reading": avg_school_size_passing_reading_scores.map("{:.0f}".format),
          "% Overall Passing": avg_school_size_overall_passing_scores.map("{:.0f}".format)})

school_size_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.8,83.9,94,96,90
Medium (1000-2000),83.4,83.9,94,97,91
Large (2000-5000),77.7,81.3,70,83,58


In [44]:
avg_school_type_math_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Math Score"]
avg_school_type_reading_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Reading Score"]
avg_school_type_passing_math_scores = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Math"]
avg_school_type_passing_reading_scores = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Reading"]
avg_school_type_overall_passing_scores = per_school_summary_df.groupby(["School Type"]).mean()["% Overall Passing"]

In [45]:
# Assemble into DataFrame.
school_type_summary_df = pd.DataFrame({
          "Average Math Score" : avg_school_type_math_scores.map("{:.1f}".format),
          "Average Reading Score": avg_school_type_reading_scores.map("{:.1f}".format),
          "% Passing Math": avg_school_type_passing_math_scores.map("{:.0f}".format),
          "% Passing Reading": avg_school_type_passing_reading_scores.map("{:.0f}".format),
          "% Overall Passing": avg_school_type_overall_passing_scores.map("{:.0f}".format)})

school_type_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.5,83.9,94,97,90
District,77.0,81.0,67,81,54
